In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pylab as plt
%matplotlib inline
from scipy import optimize

from tyssue.core.sheet import Sheet
from tyssue.config.json_parser import load_default

from tyssue.geometry.sheet_geometry import SheetGeometry as geom
from tyssue.dynamics.apoptosis_model import SheetApoptosisModel as model
from tyssue.dynamics.sheet_vertex_model import SheetModel as basemodel
from tyssue.solvers.sheet_vertex_solver import Solver as solver

#from tyssue.dynamics.sheet_isotropic_model import isotropic_relax

from tyssue.draw.vispy_draw import vp_view
from tyssue.topology.sheet_topology import type1_transition, remove_face

from tyssue.draw.mpl_draw import sheet_view
import tyssue.draw.mpl_draw as draw
from tyssue.stores import load_datasets

In [2]:

specs = {
    'edge': {
        'dx': 0.0, 'dy': 0.0, 'dz': 0.0,
        'length': 0.0,
        'nx': 0.0, 'ny': 0.0, 'nz': 1.0,
        'srce': 0, 'trgt': 0, 'face': 0, 'cell': 0,
        'is_apical': True,
        'is_basal': False,
        'sub_area': 0.0,
        'line_tension': 0.14,
        'basal_elasticity': 0.0,
        },
    'face': {
        'area': 1.0,
        'is_alive': True,
        'num_sides': 6,
        'perimeter': 1.0,
        'x': 0.0, 'y': 0.0, 'z': 0.0,
        'contractility': 0.0,
        },
    'cell': {
        'vol': 1.0,
        'vol_elasticity': 0.0, 
        'is_alive': True,
        'num_sides': 6,
        'perimeter': 1.0,
        'x': 0.0, 'y': 0.0, 'z': 0.0,
        },
     'settings': {
        'geometry': 'cylindrical',
        'height_axis': 'z'
        },
     'vert': {
        'is_active': True,
        'is_basal': False,
        'x': 0.0, 'y': 0.0, 'z': 0.0,
        }
    }




In [12]:
h5store = 'small_hexagonal.hf5'
datasets = load_datasets(h5store,
                         data_names=['face', 'vert', 'edge'])


coords = ['x', 'y', 'z']
datasets['cell'] = pd.DataFrame(index=datasets['face'].index,
                                columns=coords)
datasets['cell'].index.name = 'cell'

datasets['cell']['z'] = datasets['face']['z']

theta = np.arctan2(datasets['face']['y'],
                   datasets['face']['x'])

rho = np.hypot(datasets['face']['y'],
               datasets['face']['x'])

datasets['cell']['x'] = 2*rho/3 * np.cos(theta)
datasets['cell']['y'] = 2*rho/3 * np.sin(theta)

basal_idx = pd.Index(np.arange(datasets['face'].shape[0])+96,
                     name='vert')
basal_verts = pd.DataFrame(index=basal_idx, 
                           columns=datasets['vert'].columns)

datasets['vert'] = pd.concat([datasets['vert'], basal_verts])
datasets['vert'].loc[basal_idx, 'z'] = datasets['face']['z'].values
datasets['vert'].loc[basal_idx, 'x'] = rho.values/3 * np.cos(theta.values)
datasets['vert'].loc[basal_idx, 'y'] = rho.values/3 * np.sin(theta.values)
datasets['vert'].loc[basal_idx, 'is_active'] = 1


datasets['edge']['cell'] = datasets['edge']['face']


In [104]:
st_indexed = edge_df[['srce', 'trgt']].reset_index().set_index(['srce', 'trgt'], drop=False)
flipped = st_indexed.index.swaplevel(0, 1)
flipped.names = ['srce', 'trgt']

opposite = st_indexed.loc[flipped, 'edge'].values
edge_df['opposite'] = opposite
edge_df['opp_face'] = edge_df.loc[opposite, 'face'].values
edge_df['opp_cell'] = edge_df.loc[opposite, 'cell'].values



In [105]:
basal_edge_idx = pd.Index(
    np.arange(edge_df.shape[0]) +
    edge_df.index[-1], name='edge')

basal_edges = pd.DataFrame(index=basal_edge_idx,
                           columns=['srce', 'trgt', 'is_basal'])

basal_edges['srce'] = edge_df['cell'].values
basal_edges['trgt'] = edge_df['opp_cell'].values



In [106]:
edge_df['is_basal'] = 0
pd.concat([edge_df, basal_edges])[['srce', 'trgt']].tail()

,srce,trgt
edge,,
538,54,46
539,54,47
540,54,48
541,55,5
542,55,54


271

In [92]:
basal_edges = pd.DataFrame

In [6]:
datasets['vert'].tail()

,y,is_active,z,x,rho,old_idx,basal_shift,height,radial_tension
vert,,,,,,,,,
147,-0.725062,1.0,-4.072821e+00,1.750455,NaN,NaN,NaN,NaN,NaN
148,-1.339740,1.0,-7.401487e-17,1.339740,NaN,NaN,NaN,NaN,NaN
149,-0.725062,1.0,4.072821e+00,1.750455,NaN,NaN,NaN,NaN,NaN
150,-1.339740,1.0,8.145643e+00,1.339740,NaN,NaN,NaN,NaN,NaN
151,-1.087593,1.0,1.018205e+01,1.627698,NaN,NaN,NaN,NaN,NaN


In [7]:
datasets['cell'].head()

,x,y,z
cell,,,
0,3.255397,2.175186e+00,-1.018205e+01
1,3.789357,4.934325e-16,-8.145643e+00
2,3.500910,1.450124e+00,-4.072821e+00
3,3.789357,4.934325e-16,-7.401487e-17
4,3.500910,1.450124e+00,4.072821e+00


In [ ]:
class MixedGeometry(SheetGeometry):
        
    @saticmethod
    def update_vol(eptm):
        
        face_pos = eptm.upcast_face(eptm.face_df[eptm.coords])
        cell_pos = eptm.upcast_cell(eptm.cell_df[eptm.coords])

        eptm.edge_df['sub_vol'] = np.sum(
            (face_pos - cell_pos) *
            eptm.edge_df[eptm.ncoords].values, axis=1) / 6

        eptm.cell_df['vol'] = eptm.sum_cell(eptm.edge_df['sub_vol'])


In [4]:
datasets['edge']['cell'] = datasets['edge']['face']


,dz,length,dx,dy,srce,trgt,face,old_jv0,old_jv1,old_cell,nz,nx,ny,sub_area,sub_vol,line_tension,is_alive
edge,,,,,,,,,,,,,,,,,
0,-1.357607,2.702165,0.455802,-2.291472,1,0,0,56,57,0,-1.044458e+00,2.874114,1.190497e+00,1.640785e+00,1.454708e+01,1410.906092,0
1,1.357607,2.702165,1.298014,-1.942616,2,1,0,57,58,0,-6.661338e-16,0.000000,-6.661338e-16,4.710277e-16,4.176097e-15,1410.906092,0
2,1.357607,2.702165,-0.455802,2.291472,0,1,1,56,57,1,6.963050e-01,6.221837,8.250668e-01,3.157405e+00,2.799332e+01,1410.906092,1
3,-1.357607,2.702165,0.455802,2.291472,3,0,1,56,59,1,6.963050e-01,6.221837,-8.250668e-01,3.157405e+00,2.799332e+01,1410.906092,1
4,2.715214,2.715214,0.000000,0.000000,1,6,1,57,62,1,-0.000000e+00,6.221837,4.125334e-01,3.117749e+00,2.764173e+01,1410.906092,1


In [ ]:
class MixedModel(SheetModel):
    
    
    

In [24]:
sheet_specs

{'edge': {'dx': 0.0,
  'dy': 0.0,
  'dz': 0.0,
  'face': 0,
  'length': 0.0,
  'nx': 0.0,
  'ny': 0.0,
  'nz': 1.0,
  'srce': 0,
  'trgt': 0},
 'face': {'area': 1.0,
  'is_alive': True,
  'num_sides': 6,
  'perimeter': 1.0,
  'x': 0.0,
  'y': 0.0,
  'z': 0.0},
 'settings': {'geometry': 'cylindrical', 'height_axis': 'z'},
 'vert': {'basal_shift': 4.0,
  'is_active': True,
  'rho': 0.0,
  'x': 0.0,
  'y': 0.0,
  'z': 0.0}}